# Exploración

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
datos_enfermedad = pd.read_csv("data/Datos Lab 1.csv")

In [5]:
data = datos_enfermedad.copy()

In [6]:
data.head(20)

,Patient ID,Date of Service,Sex,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Blood Pressure (mmHg),Total Cholesterol (mg/dL),...,Physical Activity Level,Family History of CVD,Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Blood Pressure Category,Estimated LDL (mg/dL),CVD Risk Score,CVD Risk Level
0,isDx5313,"November 08, 2023",M,44.000,114.300,1.720,38.600,100.000,112/83,228.0,...,High,N,172.000,0.581,112.0,83.0,Hypertension Stage 1,121.0,19.880,HIGH
1,LHCK2961,20/03/2024,F,57.000,92.923,1.842,33.116,106.315,101/91,158.0,...,High,Y,184.172,0.577,101.0,91.0,Hypertension Stage 2,57.0,16.833,INTERMEDIARY
2,WjVn1699,2021-05-27,F,NaN,73.400,1.650,27.000,78.100,90/74,135.0,...,High,N,165.000,0.473,90.0,74.0,Normal,45.0,12.600,LOW
3,dCDO1109,"April 18, 2022",F,35.000,113.300,1.780,35.800,79.600,92/89,158.0,...,Moderate,Y,178.000,0.447,92.0,89.0,Hypertension Stage 1,94.0,14.920,HIGH
4,pnpE1080,01/11/2024,F,48.000,102.200,1.750,33.400,106.700,121/68,207.0,...,Low,Y,175.000,0.610,121.0,68.0,Elevated,128.0,18.870,HIGH
5,MQyB2747,25 Mar 24,M,43.000,52.700,1.850,15.400,107.700,107/61,105.0,...,High,N,185.000,0.582,107.0,61.0,Normal,43.0,10.530,INTERMEDIARY
6,DHdn8968,22 May 25,F,31.000,87.000,1.660,31.600,91.500,139/81,207.0,...,Low,Y,166.000,0.551,139.0,81.0,Hypertension Stage 1,121.0,17.410,HIGH
7,vkQL9700,"October 26, 2023",M,69.000,59.684,1.940,23.914,117.986,106/115,206.0,...,High,Y,193.981,0.608,106.0,115.0,Hypertension Stage 2,134.0,16.203,HIGH
8,bUBT9994,15/12/2023,F,89.162,85.600,1.660,31.100,83.600,103/99,222.0,...,High,N,166.000,0.504,103.0,99.0,Hypertension Stage 2,126.0,NaN,HIGH
9,nktq6689,"January 16, 2022",F,57.000,100.130,1.840,22.242,80.814,165/99,123.0,...,Low,N,183.988,0.439,165.0,99.0,Hypertension Stage 2,39.0,15.158,LOW


In [5]:
display(data.sample(20))

,Patient ID,Date of Service,Sex,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Blood Pressure (mmHg),Total Cholesterol (mg/dL),...,Physical Activity Level,Family History of CVD,Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Blood Pressure Category,Estimated LDL (mg/dL),CVD Risk Score,CVD Risk Level
1239,skGr1613,29/08/2021,M,47.0,50.343,NaN,35.191,103.605,173/114,205.0,...,Low,N,169.097,0.613,173.0,114.0,Hypertension Stage 2,133.0,19.788,HIGH
1183,baLv1033,2024-11-15,M,38.0,100.840,1.769,25.383,73.368,124/101,201.0,...,High,Y,176.911,0.415,NaN,101.0,Hypertension Stage 2,NaN,17.297,INTERMEDIARY
1622,PvcB0416,2021-08-27,M,51.0,NaN,1.917,26.257,99.630,173/70,157.0,...,Low,N,191.688,0.520,173.0,70.0,Hypertension Stage 2,65.0,19.041,INTERMEDIARY
1534,RWEP6047,03 Apr 24,F,51.0,102.100,1.900,28.300,94.100,116/61,297.0,...,High,N,190.000,NaN,116.0,61.0,Normal,229.0,17.400,HIGH
1301,gmaI8008,"April 04, 2022",F,43.0,98.200,1.740,32.400,106.000,103/98,NaN,...,Moderate,N,174.000,NaN,103.0,98.0,Hypertension Stage 2,35.0,15.970,INTERMEDIARY
609,szyX1885,02/10/2024,M,38.0,105.600,1.800,32.600,71.700,127/84,252.0,...,Moderate,Y,180.000,0.398,127.0,84.0,Hypertension Stage 1,174.0,11.600,HIGH
1059,INdI1482,11-24-2021,F,51.0,NaN,1.640,27.400,75.000,149/92,235.0,...,Low,Y,164.000,0.457,149.0,92.0,Hypertension Stage 2,128.0,17.630,HIGH
1479,NGwd5164,2021-12-31,M,33.0,74.000,1.760,23.900,77.400,118/86,165.0,...,High,Y,176.000,0.440,118.0,NaN,Hypertension Stage 1,64.0,13.980,LOW
630,Uexs7461,08-19-2025,F,33.0,59.623,1.626,25.751,75.782,170/76,259.0,...,Low,Y,162.599,0.466,170.0,76.0,Hypertension Stage 2,149.0,18.830,INTERMEDIARY
1050,Adqq7766,15 Nov 24,F,52.0,54.700,1.610,21.100,78.100,93/65,193.0,...,High,Y,161.000,0.485,93.0,65.0,Normal,84.0,14.730,INTERMEDIARY


In [6]:
data.info()

<class 'pandas.DataFrame'>
RangeIndex: 1639 entries, 0 to 1638
Data columns (total 24 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   Patient ID                    1639 non-null   str    
 1   Date of Service               1639 non-null   str    
 2   Sex                           1639 non-null   str    
 3   Age                           1571 non-null   float64
 4   Weight (kg)                   1566 non-null   float64
 5   Height (m)                    1578 non-null   float64
 6   BMI                           1586 non-null   float64
 7   Abdominal Circumference (cm)  1578 non-null   float64
 8   Blood Pressure (mmHg)         1639 non-null   str    
 9   Total Cholesterol (mg/dL)     1571 non-null   float64
 10  HDL (mg/dL)                   1557 non-null   float64
 11  Fasting Blood Sugar (mg/dL)   1585 non-null   float64
 12  Smoking Status                1639 non-null   str    
 13  Diabetes Statu

In [8]:
dict = pd.read_excel("data/DiccPacientes.xlsx")
pd.set_option('display.max_colwidth', None)
dict

,Nombre Columna,Tipo de dato,Comentarios
0,Patient ID,String,Identificador del paciente
1,Date of Service,Date,Fecha de la atención
2,Sex,String,"Sexo (Femenino, Masculino)"
3,Age,Integer,Edad
4,Weight (kg),Float,Peso
5,Height (m),Float,Altura
6,BMI,Float,Índice de masa corporal
7,Abdominal Circumference (cm),Float,Circunferencia abdominal
8,Blood Pressure (mmHg),String,"Presión sanguínea, de la forma ""<Presión arterial sistólica>/<Presión arterial diastólica>"""
9,Total Cholesterol (mg/dL),Float,Colesterol total


In [9]:
data.shape

(1639, 24)

In [10]:
data.describe()

,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Total Cholesterol (mg/dL),HDL (mg/dL),Fasting Blood Sugar (mg/dL),Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Estimated LDL (mg/dL),CVD Risk Score
count,1571.000000,1566.000000,1578.000000,1586.000000,1578.000000,1571.000000,1557.000000,1585.000000,1571.000000,1563.000000,1578.000000,1554.000000,1582.000000,1610.000000
mean,46.803186,85.666006,1.757439,28.424744,91.538861,199.043673,56.183558,117.836860,175.770082,0.522440,125.632637,82.887536,113.235896,18.227281
std,13.039479,21.712504,0.118012,7.309275,13.427985,59.388670,16.721702,32.379634,11.695880,0.085692,22.577463,15.503625,61.435291,10.767666
min,6.134000,13.261000,1.371000,4.317000,49.542000,-1.256000,0.008000,15.306000,136.498000,0.250000,49.914000,31.720000,-92.055000,-20.057000
25%,37.000000,67.100000,1.666500,22.600000,79.700000,150.000000,42.000000,92.000000,167.000000,0.453000,108.000000,71.000000,62.000000,15.150000
50%,46.000000,86.314000,1.760000,28.000000,91.200000,199.000000,56.000000,115.000000,176.000000,0.519000,125.000000,82.000000,112.000000,16.967000
75%,55.000000,104.801500,1.850000,33.963000,102.267250,250.000000,70.000000,139.000000,185.000000,0.582000,141.000000,93.000000,159.000000,18.900000
max,89.420000,158.523000,2.146000,53.028000,136.336000,385.679000,110.315000,219.667000,214.394000,0.804000,202.711000,134.066000,317.314000,114.980000


In [11]:
data.isna().sum()

Patient ID                       0
Date of Service                  0
Sex                              0
Age                             68
Weight (kg)                     73
Height (m)                      61
BMI                             53
Abdominal Circumference (cm)    61
Blood Pressure (mmHg)            0
Total Cholesterol (mg/dL)       68
HDL (mg/dL)                     82
Fasting Blood Sugar (mg/dL)     54
Smoking Status                   0
Diabetes Status                  0
Physical Activity Level          0
Family History of CVD            0
Height (cm)                     68
Waist-to-Height Ratio           76
Systolic BP                     61
Diastolic BP                    85
Blood Pressure Category          0
Estimated LDL (mg/dL)           57
CVD Risk Score                  29
CVD Risk Level                   0
dtype: int64

In [12]:
data["CVD Risk Level"].value_counts()

CVD Risk Level
HIGH            793
INTERMEDIARY    616
LOW             230
Name: count, dtype: int64

In [13]:
data["Blood Pressure Category"].value_counts()

Blood Pressure Category
Hypertension Stage 2    680
Hypertension Stage 1    527
Normal                  321
Elevated                111
Name: count, dtype: int64

In [14]:
data["Sex"].value_counts()

Sex
M    821
F    818
Name: count, dtype: int64

In [15]:
data["Physical Activity Level"].value_counts()

Physical Activity Level
High        582
Moderate    537
Low         520
Name: count, dtype: int64

In [9]:
data["Patient ID"].duplicated(keep=False)

0       False
1       False
2       False
3       False
4       False
        ...  
1634    False
1635    False
1636     True
1637    False
1638    False
Name: Patient ID, Length: 1639, dtype: bool

In [17]:
duplicados=data[data["Patient ID"].duplicated(keep=False)].sort_values("Patient ID")
duplicados

,Patient ID,Date of Service,Sex,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Blood Pressure (mmHg),Total Cholesterol (mg/dL),...,Physical Activity Level,Family History of CVD,Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Blood Pressure Category,Estimated LDL (mg/dL),CVD Risk Score,CVD Risk Level
17,AhYt1346,09-28-2020,M,41.0,71.300,1.730,23.800,107.900,139/61,253.0,...,Low,Y,173.000,NaN,139.0,NaN,Hypertension Stage 1,146.0,16.770,HIGH
1584,AhYt1346,09-28-2020,M,41.0,71.300,1.730,23.800,107.900,139/61,253.0,...,Low,Y,173.000,NaN,139.0,NaN,Hypertension Stage 1,146.0,16.770,HIGH
1227,AhYt1346,09-28-2020,M,41.0,71.300,1.730,23.800,107.900,139/61,253.0,...,Low,Y,173.000,NaN,139.0,NaN,Hypertension Stage 1,146.0,-13.090,HIGH
1117,Axab9332,2021-12-11,F,58.0,69.870,1.944,20.785,86.259,162/106,151.0,...,Moderate,N,194.450,0.444,162.0,106.0,Hypertension Stage 2,44.0,23.430,HIGH
383,Axab9332,2021-12-11,F,58.0,69.870,1.944,20.785,86.259,162/106,151.0,...,Moderate,N,194.450,0.444,162.0,106.0,Hypertension Stage 2,44.0,15.277,HIGH
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
956,zhZi8857,"December 05, 2025",F,36.0,81.200,1.750,26.500,74.000,141/76,225.0,...,Moderate,Y,175.000,0.423,141.0,NaN,Hypertension Stage 2,164.0,16.850,HIGH
727,zhZi8857,"December 05, 2025",F,36.0,81.200,1.750,26.500,74.000,141/76,225.0,...,Moderate,Y,175.000,0.423,141.0,NaN,Hypertension Stage 2,164.0,41.813,HIGH
1548,zhZi8857,"December 05, 2025",F,36.0,81.200,1.750,26.500,74.000,141/76,225.0,...,Moderate,Y,175.000,0.423,141.0,NaN,Hypertension Stage 2,164.0,16.850,HIGH
1565,zxhX5525,"November 13, 2021",M,26.0,58.953,1.688,25.286,NaN,110/114,258.0,...,High,Y,168.763,0.455,110.0,114.0,Hypertension Stage 2,170.0,15.717,HIGH


In [19]:
duplicados.iloc[5:80]

,Patient ID,Date of Service,Sex,Age,Weight (kg),Height (m),BMI,Abdominal Circumference (cm),Blood Pressure (mmHg),Total Cholesterol (mg/dL),...,Physical Activity Level,Family History of CVD,Height (cm),Waist-to-Height Ratio,Systolic BP,Diastolic BP,Blood Pressure Category,Estimated LDL (mg/dL),CVD Risk Score,CVD Risk Level
1469,BQvQ6431,09/11/2020,M,33.0,118.300,1.690,41.400,72.100,116/93,171.0,...,Moderate,N,210.554,0.427,116.0,93.0,Hypertension Stage 2,97.0,29.833,LOW
130,BQvQ6431,09/11/2020,M,33.0,118.300,1.690,41.400,72.100,116/93,171.0,...,Moderate,N,210.554,0.427,116.0,93.0,Hypertension Stage 2,97.0,17.500,LOW
1544,BQvQ6431,09/11/2020,M,33.0,118.300,1.690,41.400,72.100,116/93,171.0,...,Moderate,N,210.554,0.427,116.0,93.0,Hypertension Stage 2,97.0,17.500,LOW
846,BqZp2317,"April 15, 2025",F,72.0,57.836,1.554,24.008,115.565,157/75,211.0,...,Moderate,N,155.404,0.744,157.0,75.0,Hypertension Stage 2,100.0,16.872,INTERMEDIARY
305,BqZp2317,"April 15, 2025",F,72.0,57.836,1.554,24.008,115.565,157/75,211.0,...,Moderate,N,155.404,0.744,157.0,75.0,Hypertension Stage 2,100.0,7.050,INTERMEDIARY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
525,JsvV0478,"November 14, 2025",F,50.0,72.800,1.790,22.700,76.800,119/81,276.0,...,Moderate,Y,179.000,0.429,119.0,81.0,Hypertension Stage 1,210.0,16.010,HIGH
276,JsvV0478,"November 14, 2025",F,50.0,72.800,1.790,22.700,76.800,119/81,276.0,...,Moderate,Y,179.000,0.429,119.0,81.0,Hypertension Stage 1,210.0,16.010,HIGH
1351,KGuE6384,11-20-2021,F,33.0,105.351,1.680,28.038,109.807,146/91,288.0,...,High,Y,167.977,0.654,146.0,91.0,Hypertension Stage 2,210.0,19.138,LOW
491,KGuE6384,11-20-2021,F,33.0,105.351,1.680,28.038,109.807,146/91,288.0,...,High,Y,167.977,0.654,146.0,91.0,Hypertension Stage 2,210.0,20.668,LOW


In [20]:
dup_counts= (data["Patient ID"].value_counts()
                            .loc[lambda x: x > 1]
                            .sort_values(ascending=False))
for id_,n in dup_counts.items():
    print(f"Patient ID: {id_} → {n} apariciones")

Patient ID: oSqq7069 → 3 apariciones
Patient ID: gBFe4249 → 3 apariciones
Patient ID: AhYt1346 → 3 apariciones
Patient ID: RlsB8509 → 3 apariciones
Patient ID: RwGu5647 → 3 apariciones
Patient ID: fHqx3420 → 3 apariciones
Patient ID: YLCe2926 → 3 apariciones
Patient ID: dSiv4949 → 3 apariciones
Patient ID: pEpZ9034 → 3 apariciones
Patient ID: STpP5810 → 3 apariciones
Patient ID: DhUJ7239 → 3 apariciones
Patient ID: FTEC4446 → 3 apariciones
Patient ID: xbYu9929 → 3 apariciones
Patient ID: tCaB2910 → 3 apariciones
Patient ID: xFVW5778 → 3 apariciones
Patient ID: kijK0146 → 3 apariciones
Patient ID: Kbzi4901 → 3 apariciones
Patient ID: NGwd5164 → 3 apariciones
Patient ID: Lclg1001 → 3 apariciones
Patient ID: BQvQ6431 → 3 apariciones
Patient ID: viiE0140 → 3 apariciones
Patient ID: QJtE0701 → 3 apariciones
Patient ID: NIUR3991 → 3 apariciones
Patient ID: XZCU4466 → 3 apariciones
Patient ID: LpyK7269 → 3 apariciones
Patient ID: mrZW4662 → 3 apariciones
Patient ID: bCoz6266 → 3 apariciones
P